In [6]:
import dolfinx,ufl
from mpi4py import MPI

In [10]:
domain = dolfinx.mesh.create_rectangle(MPI.COMM_WORLD,[[-1,-1],[1,1]],[100,100])


function_space = dolfinx.fem.FunctionSpace(domain,('CG',1))
u = ufl.TrialFunction(function_space)
v = ufl.TestFunction(function_space)

i = ufl.Index()
poisson = ufl.grad(u)[i]*ufl.grad(ufl.conj(v))[i]





In [ ]:

form = dolfinx.fem.form(poisson)


In [ ]:

A = dolfinx.fem.petsc.assemble_matrix(form)


In [ ]:

A.assemble()


In [ ]:

from scipy.sparse import csr_matrix

In [ ]:

#convert to CSR
A = csr_matrix(A.getValuesCSR[::-1])

In [14]:
domain.topology.create_connectivity(domain.topology.dim - 1, domain.topology.dim)
boundary_facets = dolfinx.mesh.exterior_facet_indices(domain.topology)
boundary_dofs = dolfinx.fem.locate_dofs_topological(function_space, domain.topology.dim - 1, boundary_facets)
import numpy as np
boundary_state = np.complex128(123e123)
infinity_boundary = dolfinx.fem.dirichletbc(boundary_state, boundary_dofs, function_space)
u_boundary = dolfinx.fem.Function(function_space)
dolfinx.fem.petsc.set_bc(u_boundary.vector, [infinity_boundary])
u_boundary.x.scatter_forward()
from scipy.sparse import diags
boundary = diags(u_boundary.vector.getArray(),offsets=0,format='csr')

In [15]:
A_mat_with_boundary = A_mat +boundary

In [16]:
# Make S matrix

s_form = u*ufl.conj(v)*ufl.dx
s_form = dolfinx.fem.form(s_form)
S = dolfinx.fem.petsc.assemble_matrix(s_form)
S.assemble()
S_mat = csr_matrix(S.getValuesCSR)



In [17]:
from petsc4py import PETSc
# PETSc Stuff
petsc_A = PETSc.Mat().createAIJ(size=A.shape,csr=(A.indptr,A.indices,A.data))
petsc_A.assaemble()

petsc_S = PETSc.Mat().createAIJ(size=S.shape,csr=(S.indptr,S.indices,S.data))
petsc_S.assaemble()


In [ ]:
# SLEPc Stuff
from slepc4py import SLEPc
eig_problem = SLEPc.EPS().create(comm=MPI.COMM_WORLD)





In [ ]:
spectral_transform = SLEPc.ST().create()
spectral_transform.setType(SLEPc.ST.Type.SINVERT)
eig_problem.setST(spectral_transform)

In [ ]:

eig_problem.setTarget(0)
eig_problem.setWhichEigenpairs(SLEPc.EPS.Which.LARGEST_MAGNITUDE)




In [ ]:

eig_problem.setDimensions(10) # number of eigenvalues to find

In [ ]:

eig_problem.setProblemType(SLEPc.EPS.ProblemType.GHEP) # hermitian eigenvalue problem
eig_problem.setType(SLEPc.EPS.Type:KRYLOVSCHUR)) # set method for solving
eig_problem.setOperators(petsc_A,petsc_S) # define matrices of the problem


In [ ]:
eig_problem.solve()

In [ ]:
vr,vi = petsc_A.createVecs()
eigenvalues = []
eigenvectors = []
for i in range(eig_problem.getConverged()):
    lmbda = eig_problem.getEigenpair(i, vr, vi)
    eigenvalues.append(lmbda.real)

    eigenvector = np.array(vr.array) + 1j*np.array(vi)
    eigenvectors.append(eigenvector)

In [ ]:
# Plot the eigensolution to verify that it is correct
import pyvista


p = pyvista.Plotter()
topology, cell_types, x = dolfinx.plot.create_vtk_mesh(function_space)
grid = pyvista.UnstructuredGrid(topology, cell_types, x)


I = 0
	
grid["u"] = np.abs(eigenvectors[I])

warped = grid.warp_by_scalar('u')

p.add_mesh(warped,scalars='u')
p.show_axes()

p.show()

